<a href="https://colab.research.google.com/github/lkarjun/fastai-workouts/blob/master/Lesson_11_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
from fastai.vision.all import *
import matplotlib